In [255]:
import pandas as pd
import numpy as np
import scipy.stats as scipy
from __future__ import division

In [256]:
df = pd.read_csv("./Data/England/E0_16.csv")
df_14 = pd.read_csv("./Data/England/E0_14.csv")

In [257]:
df.columns

Index([u'Div', u'Date', u'HomeTeam', u'AwayTeam', u'FTHG', u'FTAG', u'FTR',
       u'HTHG', u'HTAG', u'HTR', u'Referee', u'HS', u'AS', u'HST', u'AST',
       u'HF', u'AF', u'HC', u'AC', u'HY', u'AY', u'HR', u'AR', u'B365H',
       u'B365D', u'B365A', u'BWH', u'BWD', u'BWA', u'IWH', u'IWD', u'IWA',
       u'LBH', u'LBD', u'LBA', u'PSH', u'PSD', u'PSA', u'WHH', u'WHD', u'WHA',
       u'VCH', u'VCD', u'VCA', u'Bb1X2', u'BbMxH', u'BbAvH', u'BbMxD',
       u'BbAvD', u'BbMxA', u'BbAvA', u'BbOU', u'BbMx>2.5', u'BbAv>2.5',
       u'BbMx<2.5', u'BbAv<2.5', u'BbAH', u'BbAHh', u'BbMxAHH', u'BbAvAHH',
       u'BbMxAHA', u'BbAvAHA', u'PSCH', u'PSCD', u'PSCA'],
      dtype='object')

In [258]:
res_13 = df.ix[:,:23]
res_13 = res_13.drop(['Div','Date'],axis=1)
res_14 = df_14.ix[:,:23]
res_14 = res_14.drop(['Div','Date'],axis=1)
table_features = df.ix[:,:7]
table_features = table_features.drop(['FTHG','FTAG','Div','Date'],axis=1)
bet_13 = df.ix[:,23:]


In [259]:
res_13.head()
feature_table = df.ix[:,:23]

In [260]:
res_13.tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
165,West Ham,Hull,1,0,H,0,0,D,L Mason,18,...,6,5,13,9,10,6,2,4,0,0
166,Bournemouth,Southampton,1,3,A,1,1,D,M Clattenburg,10,...,5,8,6,12,7,4,0,2,0,0
167,Man City,Arsenal,2,1,H,0,1,A,M Atkinson,14,...,5,1,9,13,8,1,4,2,0,0
168,Tottenham,Burnley,2,1,H,1,1,D,K Friend,30,...,9,2,10,11,11,2,1,2,0,0
169,Everton,Liverpool,0,1,A,0,0,D,M Dean,6,...,1,4,13,9,1,6,3,1,0,0


In [261]:
#Team, Home Goals Score, Away Goals Score, Attack Strength, Home Goals Conceded, Away Goals Conceded, Defensive Strength
table_13 = pd.DataFrame(columns=('Team','HGS','AGS','HAS','AAS','HGC','AGC','HDS','ADS',))

In [262]:
res_13.shape[0]

170

In [263]:
avg_home_scored_13 = res_13.FTHG.sum()*1.0 / res_13.shape[0]
avg_away_scored_13 = res_13.FTAG.sum()*1.0 / res_13.shape[0]
avg_home_conceded_13 = avg_away_scored_13
avg_away_conceded_13 = avg_home_scored_13
print "Average number of goals at home",avg_home_scored_13
print "Average number of goals away", avg_away_scored_13
print "Average number of goals conceded at home",avg_away_conceded_13
print "Average number of goals conceded away",avg_home_conceded_13


Average number of goals at home 1.61764705882
Average number of goals away 1.19411764706
Average number of goals conceded at home 1.61764705882
Average number of goals conceded away 1.19411764706


In [264]:
res_home = res_13.groupby('HomeTeam')
res_away = res_13.groupby('AwayTeam')

In [265]:
# list(res_home)[0]

In [266]:
table_13.Team = res_home.HomeTeam.all().values
table_13.HGS = res_home.FTHG.sum().values
table_13.HGC = res_home.FTAG.sum().values
table_13.AGS = res_away.FTAG.sum().values
table_13.AGC = res_away.FTHG.sum().values
table_13.head()

,Team,HGS,AGS,HAS,AAS,HGC,AGC,HDS,ADS
0,Arsenal,18,20,NaN,NaN,10,9,NaN,NaN
1,Bournemouth,16,7,NaN,NaN,12,16,NaN,NaN
2,Burnley,14,2,NaN,NaN,10,18,NaN,NaN
3,Chelsea,21,14,NaN,NaN,4,7,NaN,NaN
4,Crystal Palace,12,16,NaN,NaN,13,19,NaN,NaN


In [267]:
table_13.HAS = (table_13.HGS / 9.0) / avg_home_scored_13
table_13.AAS = (table_13.AGS / 9.0) / avg_away_scored_13
table_13.HDS = (table_13.HGC / 9.0) / avg_home_conceded_13
table_13.ADS = (table_13.AGC / 9.0) / avg_away_conceded_13
table_13.head()

,Team,HGS,AGS,HAS,AAS,HGC,AGC,HDS,ADS
0,Arsenal,18,20,1.236364,1.860974,10,9,0.930487,0.618182
1,Bournemouth,16,7,1.098990,0.651341,12,16,1.116585,1.098990
2,Burnley,14,2,0.961616,0.186097,10,18,0.930487,1.236364
3,Chelsea,21,14,1.442424,1.302682,4,7,0.372195,0.480808
4,Crystal Palace,12,16,0.824242,1.488779,13,19,1.209633,1.305051


In [268]:
feature_table = feature_table[['HomeTeam','AwayTeam','FTR','HST','AST']]
f_HAS = []
f_HDS = []
f_AAS = []
f_ADS = []
for index,row in feature_table.iterrows():
    f_HAS.append(table_13[table_13['Team'] == row['HomeTeam']]['HAS'].values[0])
    f_HDS.append(table_13[table_13['Team'] == row['HomeTeam']]['HDS'].values[0])
    f_AAS.append(table_13[table_13['Team'] == row['HomeTeam']]['AAS'].values[0])
    f_ADS.append(table_13[table_13['Team'] == row['HomeTeam']]['ADS'].values[0])
    
feature_table['HAS'] = f_HAS
feature_table['HDS'] = f_HDS
feature_table['AAS'] = f_AAS
feature_table['ADS'] = f_ADS

In [269]:
feature_table.head()

,HomeTeam,AwayTeam,FTR,HST,AST,HAS,HDS,AAS,ADS
0,Burnley,Swansea,A,3,9,0.961616,0.930487,0.186097,1.236364
1,Crystal Palace,West Brom,A,4,3,0.824242,1.209633,1.488779,1.305051
2,Everton,Tottenham,D,6,4,0.824242,0.651341,0.837438,0.961616
3,Hull,Leicester,H,5,5,0.618182,1.395731,0.465244,1.442424
4,Man City,Sunderland,H,4,3,1.167677,0.837438,1.767926,0.755556


In [270]:
def transformResult(row):
    if(row.FTR == 'H'):
        return 1
    elif(row.FTR == 'A'):
        return -1
    else:
        return 0

In [271]:
feature_table["Result"] = feature_table.apply(lambda row: transformResult(row),axis=1)

In [272]:
feature_table.tail(10)

,HomeTeam,AwayTeam,FTR,HST,AST,HAS,HDS,AAS,ADS,Result
160,Crystal Palace,Chelsea,A,2,6,0.824242,1.209633,1.488779,1.305051,-1
161,Middlesbrough,Swansea,H,4,2,0.618182,0.930487,0.651341,0.618182,1
162,Stoke,Leicester,D,7,8,0.755556,1.209633,0.744390,0.755556,0
163,Sunderland,Watford,H,4,4,0.755556,1.395731,0.372195,0.892929,1
164,West Brom,Man United,A,1,5,1.098990,1.209633,0.651341,0.549495,-1
165,West Ham,Hull,H,6,5,0.618182,1.302682,0.930487,1.167677,1
166,Bournemouth,Southampton,A,5,8,1.098990,1.116585,0.651341,1.098990,-1
167,Man City,Arsenal,H,5,1,1.167677,0.837438,1.767926,0.755556,1
168,Tottenham,Burnley,H,9,2,1.305051,0.465244,0.930487,0.480808,1
169,Everton,Liverpool,A,1,4,0.824242,0.651341,0.837438,0.961616,-1


In [273]:
X_train = feature_table[['HST','AST','HAS','HDS','AAS','ADS',]]
y_train = feature_table['Result']

In [274]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [275]:
clf = DecisionTreeClassifier()
clf2 = MultinomialNB()
clf3 = XGBClassifier()

clf3.fit(X_train,y_train)

XGBClassifier(base_score=0.5, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, nthread=-1, objective='multi:softprob', seed=0,
       silent=True, subsample=1)

In [276]:
y_pred = clf3.predict(X_train)
accuracy_score(y_pred,y_train)
scores = cross_val_score(clf2, X_train, y_train, cv=10)


In [277]:
print scores
print scores.mean()

[ 0.61111111  0.64705882  0.64705882  0.64705882  0.52941176  0.52941176
  0.70588235  0.52941176  0.64705882  0.75      ]
0.624346405229


## Adding Recent Performance

In [278]:
ht = res_13.loc[40].HomeTeam
at = res_13.loc[40].AwayTeam

In [279]:
feat_table = res_13.sort_index(ascending=False)
feat_table = feat_table[['HomeTeam','AwayTeam','FTR','FTHG','FTAG','HS','AS','HC','AC']]


In [280]:
new_fixtures = pd.DataFrame( [['Watford','Crystal Palace','D',0,0,0,0,0,0],
                             ['Arsenal','West Brom','D',0,0,0,0,0,0],
                             ['Burnley','Middlesbrough','D',0,0,0,0,0,0],
                             ['Chelsea','Bournemouth','D',0,0,0,0,0,0],
                             ['Leicester','Everton','D',0,0,0,0,0,0],
                             ['Man United','Sunderland','D',0,0,0,0,0,0],
                             ['Swansea','West Ham','D',0,0,0,0,0,0],
                             ['Hull','Man City','D',0,0,0,0,0,0],
                             ['Liverpool','Stoke City','D',0,0,0,0,0,0],
                             ['Southampton','Tottenham','D',0,0,0,0,0,0]],columns=feat_table.columns)

In [281]:
n = new_fixtures.append(feat_table,ignore_index=True)
n = n.sort_index(ascending=False)
n = n.reset_index().drop(['index'], axis=1)
n = n.sort_index(ascending=False)
feat_table.head()

,HomeTeam,AwayTeam,FTR,FTHG,FTAG,HS,AS,HC,AC
169,Everton,Liverpool,A,0,1,6,11,1,6
168,Tottenham,Burnley,H,2,1,30,5,11,2
167,Man City,Arsenal,H,2,1,14,6,8,1
166,Bournemouth,Southampton,A,1,3,10,14,7,4
165,West Ham,Hull,H,1,0,18,16,10,6


In [282]:
feat_table["pastHS"] = 0.0
feat_table["pastHC"] = 0.0
feat_table["pastAS"] = 0.0
feat_table["pastAC"] = 0.0
feat_table["pastHG"] = 0.0
feat_table["pastAG"] = 0.0



In [283]:
feat_table.head()

,HomeTeam,AwayTeam,FTR,FTHG,FTAG,HS,AS,HC,AC,pastHS,pastHC,pastAS,pastAC,pastHG,pastAG
169,Everton,Liverpool,A,0,1,6,11,1,6,0.0,0.0,0.0,0.0,0.0,0.0
168,Tottenham,Burnley,H,2,1,30,5,11,2,0.0,0.0,0.0,0.0,0.0,0.0
167,Man City,Arsenal,H,2,1,14,6,8,1,0.0,0.0,0.0,0.0,0.0,0.0
166,Bournemouth,Southampton,A,1,3,10,14,7,4,0.0,0.0,0.0,0.0,0.0,0.0
165,West Ham,Hull,H,1,0,18,16,10,6,0.0,0.0,0.0,0.0,0.0,0.0


In [309]:
k = 5
for i in range(feat_table.shape[0]-1,-1,-1):
    row = feat_table.loc[i]
    ht = row.HomeTeam
    at = row.AwayTeam
    ht_stats = feat_table.loc[i-1:-1][(feat_table.HomeTeam == ht) | (feat_table.AwayTeam == ht)].head(k)
    at_stats = feat_table.loc[i-1:-1][(feat_table.HomeTeam == at) | (feat_table.AwayTeam == at)].head(k)

    feat_table.set_value(i, 'pastHC', (ht_stats[ht_stats["AwayTeam"] == ht].sum().HC + ht_stats[ht_stats["HomeTeam"] == ht].sum().HC)/k)
    feat_table.set_value(i, 'pastAC', (at_stats[at_stats["AwayTeam"] == at].sum().HC + at_stats[at_stats["HomeTeam"] == at].sum().HC)/k)
    feat_table.set_value(i, 'pastHS', (ht_stats[ht_stats["AwayTeam"] == ht].sum().HS + ht_stats[ht_stats["HomeTeam"] == ht].sum().AS)/k)
    feat_table.set_value(i, 'pastAS', (at_stats[at_stats["AwayTeam"] == at].sum().HS + at_stats[at_stats["HomeTeam"] == at].sum().AS)/k)
    feat_table.set_value(i, 'pastHG', (ht_stats[ht_stats["AwayTeam"] == ht].sum().FTAG + ht_stats[ht_stats["HomeTeam"] == ht].sum().FTHG)/k)
    feat_table.set_value(i, 'pastAG', (at_stats[at_stats["AwayTeam"] == at].sum().FTAG + at_stats[at_stats["HomeTeam"] == at].sum().FTHG)/k)

f_HAS = []
f_HDS = []
f_AAS = []
f_ADS = []
for index,row in feat_table.iterrows():
    f_HAS.append(table_13[table_13['Team'] == row['HomeTeam']]['HAS'].values[0])
    f_HDS.append(table_13[table_13['Team'] == row['HomeTeam']]['HDS'].values[0])
    f_AAS.append(table_13[table_13['Team'] == row['HomeTeam']]['AAS'].values[0])
    f_ADS.append(table_13[table_13['Team'] == row['HomeTeam']]['ADS'].values[0])
    
feat_table['HAS'] = f_HAS
feat_table['HDS'] = f_HDS
feat_table['AAS'] = f_AAS
feat_table['ADS'] = f_ADS

/home/sibi/acad/prog_tools/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/sibi/acad/prog_tools/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [310]:
test_table = feat_table.drop(['FTHG','FTAG','HS','AS','HC','AC'],axis=1)

In [311]:
test_table["Result"] = test_table.apply(lambda row: transformResult(row),axis=1)
test_table.sort_index(inplace=True)

In [312]:
num_games = feat_table.shape[0]-1-10
num_games

159

In [313]:
X_train = test_table[['pastHS','pastHC','pastAS','pastAC','pastHG','pastAG','HAS','HDS','AAS','ADS']].loc[0:num_games]
y_train = test_table['Result'].loc[0:num_games]
X_test = test_table[['pastHS','pastHC','pastAS','pastAC','pastHG','pastAG','HAS','HDS','AAS','ADS']].loc[num_games:]
y_test = test_table['Result'].loc[num_games:]

In [314]:
X_test

,pastHS,pastHC,pastAS,pastAC,pastHG,pastAG,HAS,HDS,AAS,ADS
159,16.0,6.4,16.0,4.2,1.4,1.4,0.618182,1.302682,0.930487,1.167677
160,15.2,4.2,10.4,4.4,2.4,1.6,0.824242,1.209633,1.488779,1.305051
161,11.8,5.2,15.6,6.4,0.6,2.0,0.618182,0.930487,0.651341,0.618182
162,14.0,4.0,13.2,3.2,0.8,1.6,0.755556,1.209633,0.744390,0.755556
163,18.8,5.2,11.8,3.4,1.0,1.2,0.755556,1.395731,0.372195,0.892929
164,10.4,2.4,8.4,5.6,2.2,1.2,1.098990,1.209633,0.651341,0.549495
165,15.2,6.4,15.8,5.2,1.4,0.8,0.618182,1.302682,0.930487,1.167677
166,13.6,3.6,9.6,3.4,1.8,0.4,1.098990,1.116585,0.651341,1.098990
167,8.8,4.8,11.4,6.2,1.8,2.6,1.167677,0.837438,1.767926,0.755556
168,9.2,4.0,16.8,4.6,2.4,0.8,1.305051,0.465244,0.930487,0.480808


In [315]:
clf1 = DecisionTreeClassifier()
clf2 = MultinomialNB(alpha=3)
clf3 = XGBClassifier(n_estimators=3)
clf4 = RandomForestClassifier()
clf5 = KNeighborsClassifier(n_neighbors=5)
clf6 = LogisticRegression(solver='lbfgs',multi_class='multinomial')
clf1.fit(X_train,y_train)
clf2.fit(X_train,y_train)
clf3.fit(X_train,y_train)
clf4.fit(X_train,y_train)
clf5.fit(X_train,y_train)
clf6.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [316]:
for b in xrange(1,50):
    clf5 = KNeighborsClassifier(n_neighbors=b)
    clf5.fit(X_train,y_train)
    scores = accuracy_score(y_test,clf5.predict(X_test))
    scores, scores.mean()
    #Best n =16 to 22

# Adding home advantage

In [317]:
test_table["pastCornerDiff"] = test_table["pastHC"] - test_table["pastAC"]
test_table["pastGoalDiff"] = test_table["pastHG"] - test_table["pastAG"]
test_table["pastShotsDiff"] = test_table["pastHS"] - test_table["pastAG"]

In [319]:
test_table.drop(['pastHC','pastAS','pastAC','pastHG','pastAG'],axis=1)
X_train = test_table[['pastCornerDiff','pastGoalDiff','pastShotsDiff','HAS','HDS','AAS','ADS']].loc[0:num_games]
y_train = test_table['Result'].loc[0:num_games]
X_test = test_table[['pastCornerDiff','pastGoalDiff','pastShotsDiff','HAS','HDS','AAS','ADS']].loc[num_games:]
y_test = test_table['Result'].loc[num_games:]



In [320]:
clf1.fit(X_train,y_train)
#clf2.fit(X_train,y_train)
clf3.fit(X_train,y_train)
clf4.fit(X_train,y_train)
clf5.fit(X_train,y_train)
clf6.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [321]:
accuracy_score(y_test,clf6.predict(X_test))

0.5714285714285714

In [322]:
 from sklearn import preprocessing

In [323]:
X_normalized = preprocessing.normalize(X_train, norm='l2')
X_test_norm = preprocessing.normalize(X_test, norm='l2')

In [324]:
clf1.fit(X_normalized,y_train)
#clf2.fit(X_normalized,y_train)
clf3.fit(X_normalized,y_train)
clf4.fit(X_normalized,y_train)
clf5.fit(X_normalized,y_train)
clf6.fit(X_normalized,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [325]:
accuracy_score(y_test,clf5.predict(X_test_norm))

0.61904761904761907

In [326]:
X_test = test_table[['pastCornerDiff','pastGoalDiff','pastShotsDiff','HAS','HDS','AAS','ADS']].loc[num_games+11:]
X_test_norm = preprocessing.normalize(X_test, norm='l2')

In [341]:
y_pred = clf5.predict(X_test_norm)
y_pred.shape

(10,)

In [342]:
test_table.shape

(170, 17)

In [343]:
this_week = test_table[['HomeTeam','AwayTeam']].loc[num_games+11:]
this_week['Result']=y_pred


In [337]:
this_week

,HomeTeam,AwayTeam,Result
160,Crystal Palace,Chelsea,1
161,Middlesbrough,Swansea,-1
162,Stoke,Leicester,-1
163,Sunderland,Watford,1
164,West Brom,Man United,1
165,West Ham,Hull,1
166,Bournemouth,Southampton,-1
167,Man City,Arsenal,-1
168,Tottenham,Burnley,1
169,Everton,Liverpool,-1


In [331]:
def transformResultBack(row):
    if(row.Result == 1):
        return 'H'
    elif(row.Result == -1):
        return 'A'
    else:
        return 'D'

In [332]:
this_week["Res"] = this_week.apply(lambda row: transformResultBack(row),axis=1)

In [333]:
this_week

,HomeTeam,AwayTeam,Result,Res
160,Crystal Palace,Chelsea,1,H
161,Middlesbrough,Swansea,-1,A
162,Stoke,Leicester,-1,A
163,Sunderland,Watford,-1,A
164,West Brom,Man United,1,H
165,West Ham,Hull,1,H
166,Bournemouth,Southampton,1,H
167,Man City,Arsenal,-1,A
168,Tottenham,Burnley,1,H
169,Everton,Liverpool,0,D
